In [11]:
import time
import Queue
import threading
import urllib2
import sys
from sys import version_info
from __future__ import print_function
py3 = version_info[0] > 2


In [12]:
class timer(object):
    startTime = 0 # from what time the timer is starting
    currentTime = 0 # the current time
    previousTime = 0 # time from previous update
    elapsedTime = 0 # the difference between start and current time
    
    
    def __init__(self):
        # initialize timer
        self.startTime = time.clock()
        self.updateCurrentTime()
    
    def updateCurrentTime(self):
        self.previousTime = self.currentTime
        self.currentTime = time.clock()
        self.elapsedTime = self.currentTime - self.startTime
        
    def getCurrentTime(self, updateCurrent):
        if updateCurrent:
            self.updateCurrentTime()
            
        return self.currentTime
    
    def getElapsedTime(self, updateCurrent):
        if updateCurrent:
            self.updateCurrentTime()
            
        return self.elapsedTime   
    
    def isNextSecond(self, updateCurrent):
        if updateCurrent:
            self.updateCurrentTime()
            
        if int(self.currentTime) - int(self.previousTime) > 0:
            return True
        else:
            return False
        
    def returnTime(self):
        hours = int(self.elapsedTime) / 3600
        minutes = (int(self.elapsedTime) - (3600 * hours)) / 60
        seconds = int(self.elapsedTime) % 60
        
        # 60 -> 00:01:00
        # print hours
        #if hours < 10:
            #print('0', sep= '', end= '')
        #print(hours, ':', sep= '', end= '')

        # print minutes
        #if minutes < 10:
            #print('0', sep= '', end= '')
            
        #print(minutes, ':', sep= '', end= '')
        
        # print seconds
        #if seconds < 10:
            #print('0', sep= '', end= '')
            
        #print(seconds, sep= '')
        
        return [hours, minutes, seconds]

In [13]:
class linkObject(object):
    link = []
    lId = -1 # the link object's hopefully unique identification number
    pId = -1 # the link object's parent identification number
    baconNum = -1 # the link object's distance from the source address
    
    def __init__(self, lAdd, l, p, bN):
        self.link = lAdd
        self.lId = l
        self.pId = p
        self.baconNum = bN

In [14]:
class linkCounter(object):
    
    source = [] # source address
    dest = [] # destination address
    
    maxThreads = 0 # max number of threads at a time getting webpages
    threads = [] # contains the current threads
    
    linkQueue = [] # holds the links to be checked
    linksSearched = [] # holds links already checked
    
    printQueue = [] # holds information to be printed at appropriate time. (threads cause race conditions)
    
    q = Queue.Queue() # holds the pages
    
    pagesFound = 0
    programRunning = True
    
    # initialize program
    def __init__(self, s, d, m):
    
        self.source = s
        self.dest = d
        self.maxThreads = m
        
        # add source as first link object in queue
        self.linkQueue.append(linkObject(self.source, 0, 0, 0))
        
        pTime = timer()
        time = []
        timeAppended = 0
        
        self.printQueue.append(["Starting At:", self.source])
        self.printQueue.append(["Starting At:", self.source])
        self.printNumThreads()
            
        while(self.programRunning):
            try:
                
                # Time Display
                if(pTime.isNextSecond(True)):
                    time = pTime.returnTime()
                    timeAppended += 1
                    self.printQueue.append(time) # prints the time each second

                # Start Threads To Get More Pages
                if len(self.linkQueue) > 0: # if there is a link still in the queue
                    while len(self.threads) < self.maxThreads and len(self.linkQueue) > 0:
                        # create a new thread for next link
                        nextLink = self.linkQueue.pop(0) # pop the next link                             
                        self.linksSearched.append(nextLink) # add it to links searched queue so it can be ignored if found again
                        t = threading.Thread(target = self.getInfo, args = (nextLink,))
                        t.daemon = True
                        t.start()
                        self.threads.append(t)                 
                elif len(self.threads) == 0: # if there are no links in queue and all threads are finished
                    self.programRunning = False
                    self.printQueue.append("Program Finished Running: No Links Left To Search")
                
                # Remove Threads That Are Finished Getting Page Info
                # going backwards through list so if a thread is removed, doesn't upset where to go next
                #print("Length", len(self.threads))
                for i in range(len(self.threads) - 1, -1, -1): 
                    if not self.threads[i].isAlive():
                        self.printQueue.append(["Removing Thread:", i])
                        self.threads.pop(i)
                        
                # Print Contents Of Print Queue
                # anything that a thread wants to print needs to be sent to the printQueue
                # to avoid any issues with threads attempting to print simultaneously
                while len(self.printQueue) > 0:
                    print("PQ:", self.printQueue.pop(0))
                    sys.stdout.flush()
                    
            except KeyboardInterrupt:
                break
            
        self.printNumThreads()
        print(timeAppended)
        
    def printNumThreads(self):
        self.printQueue.append(["Num Active Threads", len(self.threads)])
        
    # returns true if the text is a link 
    # doesn't handle all cases i believe
    def isLink(self, text):
        if len(text) > 7:
            if text[0] == 'h':
                if text[1] == 't':
                    if text[2] == 't':
                        if text[3] == 'p':
                            if text[4] == ':':
                                if text[5] == '/':
                                    if text[6] == '/':
                                        return True

        if len(text) > 8:
            if text[0] == 'h':
                if text[1] == 't':
                    if text[2] == 't':
                        if text[3] == 'p':
                            if text[4] == 's':
                                if text[5] == ':':
                                    if text[6] == '/':
                                        if text[7] == '/':
                                            return True

        if len(text) > 4:
            if text[0] == 'w':
                if text[1] == 'w':
                    if text[2] == 'w':
                        if text[3] == '.':
                            return True
    
        return False

    # checks to see if the link is already on either list
    def linkAlreadyAdded(self, link):
        for i in self.linksSearched:
            if i.link == link:
                return True
            
        for i in self.linkQueue:
            if i.link == link:
                return True
            
        return False

    def getInfo(self, l):
        attempts = 0
        opened = False
        self.printQueue.append(["Checking:", l.link])
     
        if True:#try:
            while attempts < 5 and not opened:
                try:
                    if attempts > 0:
                        self.printQueue.append(["Attempt:", attempts + 1, "For:", l.link])
                    fin = urllib2.urlopen(''.join(l.link))
                    opened = True
                except:
                    attempts += 1
                    time.sleep(5)
                
            if not opened:
                self.printQueue.append(["Could Not Access:", l.link, "Out Of Attempts"])
                return -1
            info = fin.read()
            fin.close()
            self.q.put(info)
            self.pagesFound += 1
            
            i = 0
            while i in range(len(info)):

                quote = []
                j = 1

                # if opening quote is found
                if info[i] == "\"":

                    # find everything within quote
                    # end if end of info reached or end quote is found
                    while i + j < len(info) and info[i + j] != "\"":
                        quote.append(info[i + j])

                        j += 1

                    # if it's a link
                    if self.isLink(quote) == True:
                        self.printQueue.append(["Found A Link:", ''.join(quote)])
                        if ''.join(quote) == self.dest:
                            self.printQueue.append(["Target Found:", ''.join(quote)])
                            self.printQueue.append("Path:")
            
                            self.programRunning = False
                            
                            #self.printPath(linkObject(quote, len(self.linkQueue), l.pos, l.baconNumber + 1))
                            return True

                        # if link is not already on list
                        if self.linkAlreadyAdded(quote) == False:
                            self.linkQueue.append(linkObject(''.join(quote), len(self.linkQueue), l.pId, l.baconNum + 1))
                            #print "Adding:", l.baconNumber + 1, ''.join(quote)
                            #sys.stdout.flush()

                i += j
            
            return True

        #except:
            self.printQueue.append(["Could Not Access:", l.link])

In [15]:
test = linkCounter("https://www.reddit.com", "https://www.reddit.com/r/funny/wiki/index", 5)

PQ: ['Starting At:', 'https://www.reddit.com']
PQ: ['Starting At:', 'https://www.reddit.com']
PQ: ['Num Active Threads', 0]
PQ: ['Checking:', 'https://www.reddit.com']
PQ: [0, 0, 2]
PQ: [0, 0, 2]
PQ: [0, 0, 5]
PQ: ['Attempt:', 2, 'For:', 'https://www.reddit.com']
PQ: [0, 0, 6]
PQ: [0, 0, 7]
PQ: [0, 0, 8]
PQ: [0, 0, 10]
PQ: [0, 0, 11]
PQ: ['Attempt:', 3, 'For:', 'https://www.reddit.com']
PQ: [0, 0, 13]
PQ: [0, 0, 13]
PQ: [0, 0, 14]
PQ: [0, 0, 15]
PQ: [0, 0, 16]
PQ: [0, 0, 17]
PQ: [0, 0, 19]
PQ: [0, 0, 19]
PQ: ['Found A Link:', 'http://www.w3.org/1999/xhtml']
PQ: ['Checking:', 'http://www.w3.org/1999/xhtml']
PQ: ['Found A Link:', 'https://m.reddit.com/']
PQ: ['Checking:', 'https://m.reddit.com/']
PQ: ['Found A Link:', 'https://www.reddit.com/.rss']
PQ: ['Found A Link:', 'www.reddit.com']
PQ: ['Found A Link:', 'https://stats.redditmedia.com']
PQ: ['Found A Link:', 'https://www.reddit.com']
PQ: ['Checking:', 'https://www.reddit.com/.rss']
PQ: ['Found A Link:', 'https://events.redditmedia.c

ValueError: I/O operation on closed file